<a href="https://colab.research.google.com/github/unclepete-20/lab4-ia/blob/main/lab4_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio 4 - Inteligencia Artificial

### Integrantes:
- Pedro Arriola (20188)
- Oscar López (20679)
- Santiago Taracena (20017)
- YongBum Park (20117)

### Task 1 - Regresión lineal polinómica

#### Instrucciones: Usted usará Python a través de un Jupyter Notebook para llevar a cabo este ejercicio. Recuerde utilizar comentarios para describir lo que está haciendo en cada parte del proceso. Estará usando el juego de datos proporcionado dentro del portal. Al finalizar recuerde subir al portal un link a su repositorio en el que se pueda correr su notebook, usando https://mybinder.org/. El juego de datos proporcionado es parte de la plataforma Kaggle, dentro del cual se muestran los precios de casas en King County, Seattle. Este juego de datos incluye los precios de las casas vendidas entre mayo 2014 y mayo 2015. Nótese que el precio es dependiente de varias características como el número de habitaciones, número de baños, metros cuadrados de la sala, pisos, etc.

### Task 1.1 - Leer el archivo CSV proporcionado (kc_house_data.csv dentro de lab4_dataset.zip) y almacenarlo en un p.array para ser trabajado en el notebook.

In [1]:
# Para esto necesitaremos numpy y pandas, por lo cual se importaran las librerias
import pandas as pd
import numpy as np

# Leer el archivo CSV y almacenarlo en un DataFrame de Pandas
data = pd.read_csv("kc_house_data.csv")

# Convertir el DataFrame a un array de NumPy
array_house = data.to_numpy()

# Se imprime el array para observar su forma
array_house

array([[7129300520, '20141013T000000', 221900.0, ..., -122.257, 1340,
        5650],
       [6414100192, '20141209T000000', 538000.0, ..., -122.319, 1690,
        7639],
       [5631500400, '20150225T000000', 180000.0, ...,
        -122.23299999999999, 2720, 8062],
       ...,
       [1523300141, '20140623T000000', 402101.0, ...,
        -122.29899999999999, 1020, 2007],
       [291310100, '20150116T000000', 400000.0, ..., -122.069, 1410,
        1287],
       [1523300157, '20141015T000000', 325000.0, ...,
        -122.29899999999999, 1020, 1357]], dtype=object)

In [2]:
# Se explorara un poco sobre la forma del array
array_house.shape

(21613, 21)

In [3]:
# Se mostraran el numero dimensiones que posee el array
array_house.ndim

2

In [4]:
# Se mostraran el tamaño del array
array_house.size

453873

### Task 1.2 - Ajustar un modelo polinomial (regresión lineal) en base al juego de datos cargado de forma matricial que relaciona las variables de precio con los pies cuadrados del espacio habitable interior de los apartamentos (price - sqft living).

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

# Obtener los datos de precio y superficie de la casa
price = array_house[:, 2].reshape(-1, 1) # Esta variable representa el valor de y
sqft_living = array_house[:, 5].reshape(-1, 1) # Esta variable representa el valor de x

# Crear objeto StandardScaler y ajustarlo a los datos de sqft_living
scaler = StandardScaler()
scaler.fit(sqft_living)

# Transformar los datos de sqft_living utilizando el scaler
sqft_living_scaled = scaler.transform(sqft_living)

# Crear objeto PolynomialFeatures para generar características polinomiales de grado 2
poly = PolynomialFeatures(degree=2)

# Transformar los datos de superficie de la casa a características polinomiales de grado 2
X_poly = poly.fit_transform(sqft_living_scaled)

# Crear un modelo de regresión lineal y ajustarlo a los datos polinomiales
reg = LinearRegression().fit(X_poly, price)

# Obtener los coeficientes del modelo y el r cuadrado
coef = reg.coef_
intercept = reg.intercept_
r2 = reg.score(X_poly, price)

print("Coeficientes:", coef)
print("Intercepto:", intercept)
print("R cuadrado:", r2)

Coeficientes: [[     0.         209844.5658275   32543.08728232]]
Intercepto: [507545.05448421]
R cuadrado: 0.5327430940591443


### Task 1.3 Utilice la implementación vectorial del algoritmo de regresión lineal (descenso del gradiente visto en clase).

In [7]:
# Seleccionar las características a utilizar en la regresión lineal
X = data[["sqft_living"]].values
y = data[["price"]].values

# Se normalizan las caracteristicas
X_norm = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# Agregar columna de unos a X para término independiente
X_norm = np.hstack((np.ones((X_norm.shape[0], 1)), X_norm))

# Se inicializa theta
theta = np.zeros((2, 1))

# Se define la funcion de hipotesis
def hipotesis(X, theta):
    return X @ theta

# Se define la funcion del costo
def cost(X, y, theta):

    """
    m: es el número de ejemplos de entrenamiento.
    
    theta: Es el vector de parámetros de tamaño (n+1, 1)
    
    h: Es el vector de predicciones de tamaño (m, 1)
    
    J: Es el valor de la función de costo, que se calcula utilizando el 
    error cuadrático medio (MSE) entre las predicciones y los valores reales.
    """
        
    m = len(y)

    h = hipotesis(X, theta)

    J = (1 / (2 * m)) * np.sum((h - y) ** 2)

    return J

# Se define la funcion de descenso del gradiente
def descenso_gradiente(X, y, theta, alpha, iterations, batch_size):
    m = len(y)
    num_batches = int(np.ceil(m / batch_size)) # Se realiza por tandas

    for i in range(iterations):
        for j in range(num_batches):
            
            start = j * batch_size
            end = start + batch_size
            
            X_batch = X[start:end]
            y_batch = y[start:end]
            
            h = hipotesis(X_batch, theta)
            error = h - y_batch
            gradient = (1 / batch_size) * np.dot(X_batch.T, error)
            
            theta = theta - alpha * gradient.reshape(theta.shape)

    return theta

# Se define la funcion que permite hacer la prediccion
def prediccion(X, theta):
    X_pred = np.hstack((np.ones((X.shape[0], 1)), X))
    return hipotesis(X_pred, theta)

# Se definen los hiperparámetros
alpha = 0.01
iterations = 4000
batch_size = 128

# Se entrena el modelo
theta = descenso_gradiente(X_norm, y, theta, alpha, iterations, batch_size)

# Se puede realizar una prediccion con los parametros que se definan
size = 850
X_pred = np.array([[size]])
X_pred_norm = (X_pred - np.mean(X, axis=0)) / np.std(X, axis=0)
price = prediccion(X_pred_norm, theta)

# Se imprime el resultado de la prediccion
print(f"Precio estimado para una casa de {size} pies cuadrados: ${price[0,0]:,.2f}")
print(type(X))
print(type(y))

Precio estimado para una casa de 850 pies cuadrados: $199,201.25
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


### Task 1.4 Usando cross validation determine el grado del polinomio

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt

X = data.iloc[:, -2].values
y = data.iloc[:, -2].values

scores = []
for i in range(1, 10):
    polynomial = PolynomialFeatures(degree=i)
    x_polynomial = poly.fit_transform(X.reshape(-1, 1)) 

    linear_model = LinearRegression()
    score = cross_val_score(linear_model, x_polynomial, y, cv=7)
    